# Assignment Set 3

*Authors*: Myriam Belkhatir, Salomé Poulain, Shania Sinha

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.sparse import diags

from importlib import reload

## **3.1**: Eigenmodes of Different Drum Shapes

### **A**: Discretization of the Laplacian

This question is mostly theoretical and does not require any code. However, a visual representation of the matrix is created in the code below.

In [ ]:
import src.matrix_form_plot
reload(src.matrix_form_plot)
from src.matrix_form_plot import plot_matrix_form

# Discretization of the domain (matrix visualization)
plot_matrix_form()

In [ ]:
import src.eigenmodes
reload(src.eigenmodes)
from src.eigenmodes import EigenvalueProblem

### **B**: Eigenmodes of Differently-shaped Drums

#### **B.1**: Square with Side = L

In [ ]:
# 1. Square with side length L = 1
square_problem = EigenvalueProblem('square', [1], n=30)
square_problem.plot_domain()
eigenvalues_square, eigenvectors_square = square_problem.solve_eigen_problem()
square_problem.plot_eigenvectors()

#### **B.2**: Rectangle with Sides = L, 2L

In [ ]:
# 2. Rectangle with sides L = 1 and W = 2L = 2
rectangle_problem = EigenvalueProblem('rectangle', [1, 2], n=30)
rectangle_problem.plot_domain()
eigenvalues_rect, eigenvectors_rect = rectangle_problem.solve_eigen_problem()
rectangle_problem.plot_eigenvectors()


#### **B.3**: Circle with Diameter = L

In [ ]:
# 3. Circle with diameter L = 1
circle_problem = EigenvalueProblem('circle', [1], n=30)
circle_problem.plot_domain()
eigenvalues_circle, eigenvectors_circle = circle_problem.solve_eigen_problem()
circle_problem.plot_eigenvectors()

## **3.3**: Leapfrog Integration

In [ ]:
import src.leapfrog
reload(src.leapfrog)
from src.leapfrog import Leapfrog

### **I**: Leapfrog Integration for the Simple Harmonic Oscillator

In [ ]:
solver = Leapfrog(T=30)
solver.solve_leapfrog()
solver.plot_results_leapfrog(savefig=False)

In [ ]:
k_values = [0.5, 1.0, 1.5, 2.0, 5.0]

solver_many_k = Leapfrog(T=10)
solver_many_k.solve_leapfrog()

# Plot many positions graphs
solver_many_k.plot_position_many_k(k_values, savefig=False)

# Plot many velocities graphs
solver_many_k.plot_velocity_many_k(k_values, savefig=False)

### ***Bonus***: Leapfrog vs. Runge-Kutta (Energy Conservation Comparison)

In [ ]:
solver_compare = Leapfrog(T=50, dt=0.01)
solver_compare.plot_compare_methods(savefig=False)

### **J**: Sinusoidal Forcing of Harmonic Oscillator

In [ ]:
solver_forcing = Leapfrog(T=30)

omega_value = 0.8

solver_forcing.solve_leapfrog_with_forcing(omega=omega_value)
solver_forcing.plot_results_leapfrog(savefig=False, with_forcing=True, omega=omega_value)

In [ ]:
solver_forcing.plot_phase_space(savefig=False)